In [1]:
import numpy as np
import cv2
import re
import matplotlib
import shutil
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import os
import requests
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)



def download_image(url, folder_name, num):

    # write image to file
    reponse = requests.get(url)
    if reponse.status_code == 200:
        with open(os.path.join(folder_name, str(num)+".jpg"), 'wb') as file:
            file.write(reponse.content)

main_folder_name = 'Images'
if not os.path.isdir(main_folder_name):
    os.makedirs(main_folder_name)

players = ["lionel_messi", "cristiano_ronaldo", "charles_leclerc", "alex_morgan"]
for j in range(len(players)):
    #creating player directory
    sub_folder_name = players[j]
    if not os.path.isdir(os.path.join(main_folder_name, sub_folder_name)):
        os.makedirs(os.path.join(main_folder_name, sub_folder_name))
    player = players[j].split("_")  # #creating search string
    search_string = player[0]+"+"+player[1]+"+"+"face"

    url = f"https://www.google.com/search?q={search_string}&source=lnms&tbm=isch"
    driver.get(url)
    time.sleep(7)
    for i in range(1, 55):
        img_url = ""
        #At div 25 we do not have images.
        if i % 25 != 0:
            try:
                image_container = driver.find_element(By.XPATH,f'//*[@id="islrg"]/div[1]/div[{i}]/a[1]/div[1]/img').click()
                time.sleep(3)
                img_url = driver.find_element(By.XPATH,'//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div[3]/div[1]/a/img[1]').get_attribute("src")
                print(img_url)
                time.sleep(1)
            except:
                print("Could not click")
            try:
                if img_url !="":
                    download_image(img_url, os.path.join(main_folder_name, sub_folder_name), i)
            except:
                print("Could not download.")

In [ ]:
face_cascade = cv2.CascadeClassifier('opencv/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('opencv/haarcascades/haarcascade_eye.xml')

In [ ]:
scrapped_data = "Images/"
dataset_path = "cropped_dataset/"

In [ ]:
dataset_images = []
names = {}
cropped_image_dirs = []

if os.path.exists(dataset_path):
     shutil.rmtree(dataset_path)         #delete current contents of the folder if folder exists

for img_dir in img_dirs:
    name = img_dir.split('/')[-1]        #get celebrity name from it directory
    print(name)
    names[name] = []
    c=1
    for entry in os.scandir(img_dir):
        path = entry.path
        path_str = re.sub(r'\\', '/', path)        #replace double backslash with frontslash to get image path
        img = cv2.imread(path_str)
        print(path_str)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   #convert to greyscale
        faces_detected = face_cascade.detectMultiScale(gray, 1.3, 5)  #detect faces
        for (x,y,w,h) in faces_detected:
            gray_img_face = gray[y:y+h, x:x+w]                        #crop face in greyscale and colored image
            colored_face = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(gray_img_face)        #detect eyes to ensure image is clear
            if len(eyes) >= 2:                                        #only add image in dataset if full face with eyes visib
                new_dataset = dataset_path + name
                if not os.path.exists(new_dataset):
                    os.makedirs(new_dataset)
                    cropped_image_dirs.append(new_dataset)
                cropped_face = name + str(c) + ".jpg"
                file_path = new_dataset + "/" + cropped_face
                
                cv2.imwrite(file_path, colored_face)
                names[name].append(file_path)
                c += 1

In [ ]:
import numpy as np
from random import randint
import random

# Define the input and output directories
input_dir = "cropped_dataset/"


rotation_angles = [90, -90, 180]

for subfolder in os.listdir(input_dir):
    name = subfolder.split('/')[-1]        #get celebrity name from it directory
    print(name)
    subfolder_path = os.path.join(input_dir, subfolder)
    print(subfolder_path)
  
    for filename in os.listdir(subfolder_path):
    
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(subfolder_path, filename)
            image = cv2.imread(image_path)
            
        angle = random.choice(rotation_angles)  # Applying augmentation
        if angle == 90:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        elif angle == -90:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        elif angle == 180:
             rotated_image = cv2.rotate(image, cv2.ROTATE_180)
       
        output_filename = f"{os.path.splitext(filename)[0]}_aug_.jpg"
        output_path = os.path.join(subfolder_path, output_filename)
        names[name].append(output_path)
        cv2.imwrite(output_path, rotated_image)
        

In [1]:
!git add . 

In [2]:
!git commit -m "Data Augmentation added"

[main a08b07e] Data Augmentation added
 2 files changed, 90 insertions(+), 68 deletions(-)


In [3]:
!git push origin main

To https://github.com/Eesa-Sarosh/Sports-Player-Classifier.git
   fcfea03..a08b07e  main -> main


In [ ]:
for key, value in names.items():
    updated_value = [path.replace('\\', '/') for path in value]
    names[key] = updated_value

In [ ]:
names

In [ ]:
import numpy as np
import pywt
import cv2    

def w2d(img, mode='haar', level=1): #Function to extract features from an image

    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [ ]:
img = cv2.imread('cropped_dataset/cristiano_ronaldo/cristiano_ronaldo4.jpg')
im_har = w2d(img,'db1',5)
plt.imshow(im_har, cmap='gray')

In [ ]:
names_map = {}
count = 0
for celebrity_name in names.keys():
    names_map[celebrity_name] = count
    count = count + 1
names_map

In [ ]:
X, y = [], []
for celebrity_name, img_path in names.items():
    for file in img_path:
        img = cv2.imread(file)
        if img is not None:
            raw_img = cv2.resize(img, (32, 32)) #setting image size to 32x32
            img_har = w2d(img,'db1',5) #getting wavelet transformed image
            wave_img = cv2.resize(img_har, (32, 32))
            combined_img = np.vstack((raw_img.reshape(32*32*3,1),wave_img.reshape(32*32,1))) #Stacking the image one to another for input
            X.append(combined_img)
            y.append(names_map[celebrity_name])     

In [ ]:
X = np.array(X).reshape(len(X),4096).astype(float)
X.shape

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Preprocess the data scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#train the SVM model
svc = SVC(kernel='linear')
svc.fit(X_train_scaled, y_train)

# Assess the model performance on the test set
accuracy = svc.score(X_test_scaled, y_test)
accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

# Preprocess the data - scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1))

# Instantiate logistic regression model
logistic_regression = LogisticRegression(max_iter=1000,C=.5, solver='lbfgs')

# Train the model
logistic_regression.fit(X_train_scaled, y_train)

# Predict using the trained model
predictions = logistic_regression.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Test accuracy: {accuracy}')

report = classification_report(y_test, predictions)
print("Classification Report:")
print(report)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Creating Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  

# Training the Random Forest classifier
rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

# Calculating accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
# Split the data into training and testing sets
conf_matrix = confusion_matrix(y_test, predictions)
# Plot confusion matrix using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
!pip install joblib
import joblib 
joblib.dump(logistic_regression, 'saved_model.pkl') 

In [ ]:
import json
with open("class_dictionary.json","w") as f:
    f.write(json.dumps(names_map))